### sklearn.neural_network.MLPClassifier
### sklearn.neural_network.MLPRegressor
- 입력층 : 특성치(X)
- 출력층 : 레이블(y)

#### 주요 Hyperparameter
- hidden_layer_sizes : 은닉층 수
- 은닉층의 노드 수
- learning_rate : 학습률(epoch)
- activation : 활성화 함수 - sigmoid, relu 등
- solver : 고속 옵티마이저 : 오차를 줄이기 위해 해를 찾는 파라미터. SGD, Adam 등

##### MLPClassifier / MLPRegressor(hidden_layer_sizes=100, activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, ealry_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)

# 분석 코드 - Classification

In [1]:
# 라이브러리 및 데이터 로드
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error

df = pd.read_csv('../input/big-data-certification-study/breast-cancer-wisconsin.csv', encoding='utf-8')
df.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
# 데이터 분리
X=df.drop(columns=['code','Class'])
y=df[['Class']]
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42)

In [3]:
# 정규화
scaler=MinMaxScaler()
scaler.fit(X_train)
mm_X_train=scaler.transform(X_train)
mm_X_test=scaler.transform(X_test)

In [4]:
# 모델 적용
model=MLPClassifier()
model.fit(mm_X_train, y_train)
pred_train=model.predict(mm_X_train)
model.score(mm_X_train,y_train)

0.974609375

In [5]:
# 혼동행렬, 분류예측 보고서
cm_train=confusion_matrix(y_train,pred_train)
cfr_train=classification_report(y_train,pred_train)
print('혼동행렬 :\n',cm_train,
      '\n\n\n분류예측 보고서 :\n',cfr_train)

혼동행렬 :
 [[328   5]
 [  8 171]] 


분류예측 보고서 :
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       333
           1       0.97      0.96      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512



In [6]:
# 모델 적용
pred_test=model.predict(mm_X_test)
model.score(mm_X_test, y_test)

0.9590643274853801

In [7]:
# 혼동행렬, 분류예측 보고서
cm_test=confusion_matrix(y_test,pred_test)
cfr_test=classification_report(y_test,pred_test)
print('혼동행렬 :\n',cm_test,'\n\n\n분류예측 보고서 :\n',cfr_test)

혼동행렬 :
 [[106   5]
 [  2  58]] 


분류예측 보고서 :
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [8]:
# Hyperparameter Tuning
# Grid Search
param_g = {'hidden_layer_sizes':[10,30,50,100], 'solver':['sgd','adam'],'activation':['tanh','relu']}
grid= GridSearchCV(MLPClassifier(), param_g, cv=5, return_train_score=True)
grid.fit(mm_X_train, y_train)

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'],
                         'hidden_layer_sizes': [10, 30, 50, 100],
                         'solver': ['sgd', 'adam']},
             return_train_score=True)

In [9]:
print('Best Parameter :', grid.best_params_)
print('Best Score :',round(grid.best_score_,4))
print('Test Score :',round(grid.score(mm_X_test, y_test),4))

Best Parameter : {'activation': 'relu', 'hidden_layer_sizes': 50, 'solver': 'adam'}
Best Score : 0.9765
Test Score : 0.9591


In [10]:
# Randomized Search
param_r={'hidden_layer_sizes':randint(low=10,high=100), 'solver':['sgd','adam'],'activation':['tanh','relu']}
random=RandomizedSearchCV(MLPClassifier(), param_distributions=param_r, cv=5, n_iter=10, return_train_score=True)
random.fit(mm_X_train, y_train)

RandomizedSearchCV(cv=5, estimator=MLPClassifier(),
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'hidden_layer_sizes': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8ee25929d0>,
                                        'solver': ['sgd', 'adam']},
                   return_train_score=True)

In [11]:
print('Best Parameter :', random.best_params_) 
print('Best Score :',round(random.best_score_,4)) 
print('Test Score :',round(random.score(mm_X_test, y_test),4))

Best Parameter : {'activation': 'relu', 'hidden_layer_sizes': 79, 'solver': 'adam'}
Best Score : 0.9726
Test Score : 0.9591


# 분석 코드 - Regression

In [12]:
df2=pd.read_csv('../input/big-data-certification-study/house_price.csv', encoding='utf-8') 
df2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [13]:
# 데이터 분리 
X=df2.drop(columns=['house_value']) 
y=df2[['house_value']] 
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [14]:
# 정규화 
scale=MinMaxScaler() 
scale.fit(X_train) 
ms_X_train=scale.transform(X_train) 
ms_x_test=scale.transform(X_test)

In [15]:
# 모델 적용 
model_r=MLPRegressor() 
model_r.fit(ms_X_train, y_train) 
pred_x=model_r.predict(ms_X_train) 
model_r.score(ms_X_train, y_train)

-2.5047624032906692

In [16]:
pred_y=model_r.predict(ms_x_test) 
model_r.score(ms_x_test,y_test)

-2.465133935116116

In [17]:
# RMSE 
rmse_train=np.sqrt(mean_squared_error(y_train,pred_x)) 
rmse_test=np.sqrt(mean_squared_error(y_test,pred_y)) 
print('Train RMSE :', round(rmse_train), '\nTest  RMSE :', round(rmse_test))

Train RMSE : 178682 
Test  RMSE : 177960


In [18]:
mlpr=MLPRegressor(hidden_layer_sizes=(64,64,64),  # 3개 은닉층과 각각 64개의 노드 설정
                  activation='relu',
                  random_state=1,max_iter=2000) 
mlpr.fit(ms_X_train, y_train) 
pred_x=mlpr.predict(ms_X_train) 
mlpr.score(ms_X_train, y_train)

# MLP 회귀모델은 탐색으로 최적 조합을 찾기 힘들기 때문에 임의 설정

0.5930441651564293

In [19]:
pred_y=mlpr.predict(ms_x_test) 
mlpr.score(ms_x_test,y_test)

0.6059090052608731

In [20]:
# RMSE 
rmse_train=np.sqrt(mean_squared_error(y_train,pred_x)) 
rmse_test=np.sqrt(mean_squared_error(y_test,pred_y)) 
print('Train RMSE :', round(rmse_train), '\nTest  RMSE :', round(rmse_test))

Train RMSE : 60887 
Test  RMSE : 60015
